# Intro to managing assignments

All of the work managing assignments is completed in the nbgrader `<course_directory>`, while logged in as the user "instructor".  For the 2015-fall-big_data class, the `<course_directory>` is `/home/instructor/nbgrader/courses/2015-fall-big_data`.  This IPython notebook, `manage_assignments.ipynb`, should be run from the `<course_directory>`.  Do not try to run it in any other directory.

This notebook contains:

- instructions on initializing the course's gradebook database.
- an overview of the process of making, releasing, collecting, and grading assignments.
- tips for troubleshooting problems with assignments.

# Initialize gradebook database

A given course's students, assignments, and grades are stored in that course's gradebook database.  This database file is stored at `<course_directory>/gradebook.db`.  Before you do any work with assignments, you need to initialize the gradebook database by adding students and assignments to it.

## Add students to the gradebook

Before you do anything else, you'll need to add all your students to the gradebook.

Assumptions:

- as long as the "source" directory is in the same directory as this page, everything should be fine.
- you'll need to make one line per student.  If you forget a student and then try to grade, there will be errors, so enter all your students right from the start.

In [1]:
# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook

# connect to the database.
gb = Gradebook("sqlite:///gradebook.db")

# add some students to the database
# template:
# gb.add_student( "<unix_username>", first_name = "<first_name>", last_name = "<last_name>" )
gb.add_student( "jonathanmorgan", first_name = "Jonathan", last_name = "Morgan" )

Student<jonathanmorgan>

## Add an assignment to the gradebook

Use the code below to add an assignment to the gradebook.  You should just need to set the assignment_name and assignment_due_date fields.  More notes on other options TK.

Assumptions:

- as long as the "source" directory is in the same directory as this page, everything should be fine.
- should not need to list out the ipython notebooks in the assignment - you are just telling it the name of the assignment (which is also the name of the folder in "source" in which the assignment's notebooks live).

In [2]:
import os

# remove an existing database
#if os.path.exists("gradebook.db"):
#    os.remove("gradebook.db")

# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook
gb = Gradebook("sqlite:///gradebook.db")

#==================================
# set up assignment call variables
#==================================

# assignment_name - the name of the folder inside the "source" folder that contains
#    the ipython notebooks for a given assignment.
# example: assignment_name = "08. Machine Learning"
assignment_name = "06. Networks"

# assignment_due_date - the date the assignment is due, in format "YYYY-MM-DD HH:MM:SS:MMMMMM TZ"
# example: assignment_due_date = "2015-02-01 15:00:00.000000 PST"
assignment_due_date = "2015-10-01 15:00:00.000000 EST"

# add the assignment to the database and specify a due date
gb.add_assignment( assignment_name, duedate = assignment_due_date )

Assignment<06. Networks>

<hr />

# Assignment workflow after adding to gradebook

Once you've run the code above to add an assignment to the gradebook, you'll use the `nbgrader` command to:

- make a student version of the assignment
- release it to the students
- check for submissions
- gather submissions for grading
- autograde all assignments (even those with no auto-grade questions, just so you can use the form grade app).
- start the formgrade app to grade assignments that can not be auto-graded.

Details on each step of this process, using an example assignment named "06. Networks":

## 1) render student version - `nbgrader assign "<assignment>"`

Use the `nbgrader assign "<assignment>"` command to render the student version of the notebook:

    nbgrader assign "06. Networks"
    
- Example output:
    
        $ nbgrader assign "06. Networks"
        Networks" --IncludeHeaderFooter.header=source/Style\ Guide.ipynb
        [AssignApp | INFO] Converting notebook source/./06. Networks/networks_exercise.ipynb to notebook
        [AssignApp | INFO] Writing 11791 bytes to release/./06. Networks/networks_exercise.ipynb
        [AssignApp | INFO] Setting destination file permissions to 644  
        
After the first time you do this, to re-render, then overwrite the existing student version of the assignment:
    
    nbgrader assign "06. Networks" --force
        
- Example output:
    
        $ nbgrader assign "06. Networks" --force
        [AssignApp | WARNING] Removing existing assignment: release/06. Networks
        [AssignApp | INFO] Converting notebook source/./06. Networks/networks_exercise.ipynb to notebook
        [AssignApp | INFO] Writing 11791 bytes to release/./06. Networks/networks_exercise.ipynb
        [AssignApp | INFO] Setting destination file permissions to 644  
        
- Example output if you forget the `--force`:

        $ nbgrader assign "06. Networks"
        [AssignApp | INFO] Skipping existing assignment: release/06. Networks

If you have a header (we don't at the moment):
    
    nbgrader assign "06. Networks" --IncludeHeaderFooter.header=source/Style\ Guide.ipynb
    
Running `nbgrader assign "<assignment>"` on an assignment renders student versions of the assignment's notebooks, then  places the student versions in a directory named the same as the assignment in the `<course_folder>/release` folder.

## 2) release student version - `nbgrader release "<assignment>"`

Use the `nbgrader release "<assignment>"` command to place the assignment in the `exchange` folder:

    nbgrader release "08. Machine Learning"
    
- Example output:

        $ nbgrader release "06. Networks"
        [ReleaseApp | INFO] Source: /home/instructor/nbgrader/courses/2015-fall-big_data/release/06. Networks
        [ReleaseApp | INFO] Destination: /srv/nbgrader/exchange/2015-fall-big_data/outbound/06. Networks
        [ReleaseApp | INFO] Released as: 2015-fall-big_data 06. Networks
        
After the first time you do this, to overwrite the existing copy of the assignment in the exchange:
    
    nbgrader release "06. Networks" --force
    
- Example output:

        $ nbgrader release "06. Networks" --force
        [ReleaseApp | INFO] Overwriting files: 2015-fall-big_data 06. Networks
        [ReleaseApp | INFO] Source: /home/instructor/nbgrader/courses/2015-fall-big_data/release/06. Networks
        [ReleaseApp | INFO] Destination: /srv/nbgrader/exchange/2015-fall-big_data/outbound/06. Networks
        [ReleaseApp | INFO] Released as: 2015-fall-big_data 06. Networks

- Example output if you forget the `--force`:

        $ nbgrader release "06. Networks"
        [ReleaseApp | ERROR] Destination already exists, add --force to overwrite: 2015-fall-big_data 06. Networks
        
Releasing an assignment places all of that assignment's notebooks in `<exchange_folder>/<class>/outbound/<assignment>/`.  So, for the default exchange directory path ("/srv/nbgrader/exchange"), class "2015-fall-big_data" and assignment "06. Networks", the assignment's notebooks would be placed in:

    /srv/nbgrader/exchange/2015-fall-big_data/outbound/06. Networks/

## 3) STUDENT - Find assignments - `nbgrader list`

Once the assignment has been released, the student can see an assignment has been released by either:

- looking in the "Released assignments" section of the "Assignments" tab.
    
- OR checking the output of the nbgrader `nbgrader list` command:

        nbgrader list
        
    - Example output:
    
            $ nbgrader list
            [ListApp | INFO] Released assignments:
            [ListApp | INFO] 2015-fall-big_data 06. Networks


## 4) STUDENT - Fetch assignment - `nbgrader fetch "<assignment>"`

Students can download a released assignment to work on it by either:

- clicking on the "Fetch" button next to the asignment in the **"Released assignments"** section of the **"Assignments"** tab.

- OR running the `nbgrader fetch "<assignment>"` command:
    
        nbgrader fetch "08. Machine Learning"
     
    - Example output:
    
            $ nbgrader fetch "06. Networks"
            [FetchApp | INFO] Source: /srv/nbgrader/exchange/2015-fall-big_data/outbound/06. Networks
            [FetchApp | INFO] Destination: /home/jmorgan/Downloads/06. Networks
            [FetchApp | INFO] Fetched as: 2015-fall-big_data 06. Networks
            
If you use the `nbgrader fetch` command, your assignments will be downloaded into the folder in which you run the command.  The "Fetch" button will always download your assignments into your home folder.  If you don't want the assignments downloaded directly into your home folder, you must use the command line command rather than the Assignments tab, as the Assignments tab doesn't know to look anywhere other than your home folder for assignments.
            


## 5) STUDENT - Submit assignment - `nbgrader submit "<assignment>"`

Once the student has completed the notebook(s) that make up the assignment, they can turn the assignment in by either:

- clicking on the "Submit" button next to the assignment in the "Downloaded assignments" section of the "Assignments" tab.
- OR running the `nbgrader submit "<assignment>"` command:
    
        nbgrader submit "06. Networks"
            
If you used the `nbgrader fetch "<assignment>"` command to fetch an assignment to a folder other than your home folder, you will have to use the `nbgrader submit "<assignment>"` command in the folder where you downloaded the assignment to submit it, again because the Assignments tab doesn't know to look anywhere other than your home folder for assignments.

After a student submits their assignment, the updated notebook(s) are stored in `<exchange_folder>/inbound`, in a folder named `<student_user>+<assignment>+<timestamp>` (example: `jmorgan+06. Networks+2015-08-29 23:33:29 UTC`).  Inside, each notebook submitted by the student for the assignment is stored, as well as a file named `timestamp.txt` that contains the same time stamp as is appended to the folder name.

## 6) List assignments with submissions - `nbgrader list --inbound`

To see which assignments have submissions which can be graded, in the courses directory, the instructor can use the `nbgrader list` command:

    nbgrader list --inbound
        
- Example output (course, then username, then assignment, then submission date):

        $ nbgrader list --inbound
        [ListApp | INFO] Submitted assignments:
        [ListApp | INFO] 2015-fall-big_data jmorgan 06. Networks 2015-08-29 23:33:29 UTC


## 7) Collect assignments - `nbgrader collect "<assignment>"`

To collect submitted assignments for grading, after you see submissions in `nbgrader list --inbound`, use the `nbgrader collect "<assignment>"` command to collect assignments back into the instructor's grading area from the exchange folder:

     nbgrader collect "06. Networks"
        
- Example output:

        $ nbgrader collect "06. Networks"
        [CollectApp | INFO] Collecting submission: jmorgan 06. Networks
        
After running `nbgrader collect "<assignment>"`, the collected assignments are stored in the `<course_directory>/submitted` folder.  Inside, each student has a directory that matches their unix username.

Each student's directory contains a folder for each assignment.  Inside each assignment folder are the notebooks that were collected and a timestamp file that holds the date and time the assignment was submitted.

## 8) Auto-grade assignments - `nbgrader autograde "<assignment>"`

After you have collected some assignments, you will autograde them using the `nbgrader autograde "<assignment>"` command:

    nbgrader autograde "06. Networks"
    
- Example output:

        [AutogradeApp | INFO] Copying submitted/jmorgan/06. Networks/timestamp.txt -> autograded/jmorgan/06. Networks/timestamp.txt
        [AutogradeApp | INFO] SubmittedAssignment<06. Networks for jmorgan> submitted at 2015-08-29 23:33:29
        [AutogradeApp | INFO] Overwriting files with master versions from the source directory
        [AutogradeApp | INFO] Sanitizing submitted/jmorgan/06. Networks/networks_exercise.ipynb
        [AutogradeApp | INFO] Converting notebook submitted/jmorgan/06. Networks/networks_exercise.ipynb to notebook
        [AutogradeApp | INFO] Writing 8008 bytes to autograded/jmorgan/06. Networks/networks_exercise.ipynb
        [AutogradeApp | INFO] Autograding autograded/jmorgan/06. Networks/networks_exercise.ipynb
        [AutogradeApp | INFO] Converting notebook autograded/jmorgan/06. Networks/networks_exercise.ipynb to notebook
        [AutogradeApp | INFO] Executing notebook with kernel: python2
        [AutogradeApp | INFO] Writing 10097 bytes to autograded/jmorgan/06. Networks/networks_exercise.ipynb
        [AutogradeApp | INFO] Setting destination file permissions to 444

You must auto-grade all assignments, even those without auto-graded questions, since the form grade app also depends on the output of this command.

Once assignments are auto-graded, they are stored in the `<course_directory>/autograded` folder.  Inside, each student has a directory that matches their unix username.

Each student's directory contains a folder for each assignment.  Inside each assignment folder are the notebooks that were collected and a timestamp file that holds the date and time the assignment was auto-graded.

## 9) Start the Formgrade App - `nbgrader formgrade`

Before you start the formgrade app, you need to have its properties configured in the nbgrader_config.py file that is in the same folder as this file.  For more details, see [http://nbgrader.readthedocs.org/en/stable/user_guide/11_jupyterhub_config.html#configuring-nbgrader-formgrade](http://nbgrader.readthedocs.org/en/stable/user_guide/11_jupyterhub_config.html#configuring-nbgrader-formgrade).

To start the formgrade app while logged in as the instructor user:

- open a screen session in which you'll let the app run:

        screen
        
- make sure you are in the `<course_directory>`.

        pwd

- export the same `CONFIGPROXY_AUTH_TOKEN` environment variable value as is used to start the jupyterhub itself:

        export CONFIGPROXY_AUTH_TOKEN='<CONFIGPROXY_AUTH_TOKEN>'
    
- run `nbgrader formgrade`:

        nbgrader formgrade
        
    - example output:
    
            $ nbgrader formgrade
            [FormgradeApp | INFO] Proxying /hub/nbgrader/2015-fall-big_data --> http://127.0.0.1:9000
            [FormgradeApp | INFO] Serving MathJax from /usr/local/lib/python3.4/dist-packages/notebook/static/components/MathJax/
            [FormgradeApp | INFO] Form grader running at http://127.0.0.1:9000/
            [FormgradeApp | INFO] Use Control-C to stop this server
            * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)

For more information on troubleshoting, see "`nbgrader formgrade` issues" in the troubleshooting section below.

<hr/>

# Troubeshooting

## `nbgrader formgrade` issues

### CONFIGPROXY_AUTH_TOKEN mismatch between jupyterhub and formgrade

Example output for CONFIGPROXY_AUTH_TOKEN mismatch between server jupyterhub and formgrade:

    $ nbgrader formgrade
    [FormgradeApp | INFO] Proxying /hub/nbgrader/2015-fall-big_data --> http://127.0.0.1:9000
    Traceback (most recent call last):
      File "/usr/local/bin/nbgrader", line 11, in <module>
        sys.exit(main())
      File "/usr/local/lib/python3.4/dist-packages/nbgrader/apps/nbgraderapp.py", line 232, in main
        NbGraderApp.launch_instance()
      File "/usr/local/lib/python3.4/dist-packages/jupyter_core/application.py", line 267, in launch_instance
        return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
      File "/usr/local/lib/python3.4/dist-packages/traitlets/config/application.py", line 592, in launch_instance
        app.start()
      File "/usr/local/lib/python3.4/dist-packages/nbgrader/apps/nbgraderapp.py", line 225, in start
        super(NbGraderApp, self).start()
      File "/usr/local/lib/python3.4/dist-packages/jupyter_core/application.py", line 256, in start
        self.subapp.start()
      File "/usr/local/lib/python3.4/dist-packages/nbgrader/apps/formgradeapp.py", line 125, in start
        parent=self)
      File "/usr/local/lib/python3.4/dist-packages/nbgrader/auth/hubauth.py", line 102, in __init__
        raise Exception('Error while trying to add JupyterHub route. {}: {}'.format(response.status_code, response.text))
    Exception: Error while trying to add JupyterHub route. 403: 

If you see something like this, make sure that you are explicitly setting the CONFIGPROXY_AUTH_TOKEN for the jupyterhub server (preferably in the `c.JupyterHub.proxy_auth_token` variable in `jupyterhub_config.py`), and that you are exporting that same value before starting the formgrade app.

### "attempt to write a readonly database"

If you see an error like this:

    sqlalchemy.exc.OperationalError: (sqlite3.OperationalError) attempt to write a readonly database [SQL: 'INSERT INTO api_tokens (hashed, prefix, user_id) VALUES (?, ?, ?)'] [parameters: ('sha512:16384:09072f984a3396f7:7d01c1f9e2e9bef9abdf71342eac9feaa04757c5845e97dbbcc3ed34f2a49dd92bfc741ad7776cdba87a2fc33410b6bc6be1d0d983cb7c05e9debd45455554a0', 'ab39', 6)]
    
This means your jupyterhub.sqlite database is not able to be written to by the user used to start the formgrade application.  Try adjusting the permissions so that the user who starts formgrade can write to the database.

### "unable to open database file" api_tokens

If you see the following in the output from `nbgrader formgrade`:

    sqlalchemy.exc.OperationalError: (sqlite3.OperationalError) unable to open database file [SQL: 'INSERT INTO api_tokens (hashed, prefix, user_id) VALUES (?, ?, ?)'] [parameters: ('sha512:16384:19f732de1639d068:c517288d4f3716af666657325274c91812ad97e8252f5cf218db2c9f62c4c698a3762f62f27d8fe761f781740d2b525ce58a8a33d91308325761950524dd2aba', 'a151', 6)]
    
This is likely because the folder that contains your `jupyterhub.sqlite` database file is not writeable by the user used to start the formgrade application (sqlite writes temp files in the directory where the database it is using lives).  Try adjusting the permissions on the directory that contains `jupyterhub.sqlite` so that the user who starts formgrade can write to it.

## notebook issues

### nbgrader solution can not be in a normal cell

If, when you run `nbgrader assign`, you get the error:

    RuntimeError: Solution region detected in a non-solution cell; please make sure all solution regions are within solution cells
        
It could be because you created your notebook with cells that contain nbgrader (for example, `### BEGIN SOLUTION ###` and `### END SOLUTION ###`), but aren't specified as nbgrader cells using the "Create Assignment" cell toolbar.  Make sure that all cells that have solutions are specified correctly as Assignment cells using the toolbar.

### grade cell point value must be greater than 0

If, when you run `nbgrader assign`, you get the error:

    RuntimeError: Point value for grade cell correct_sum_of_squares is invalid:
        
Make sure that each of your graded Assignment cells has a point value that is greater than 0.  It can be a decimal like 0.5, but 0 is invalid.

### make sure you run code cells that just contain a function definition before calling the function

If you don't, and then you try to run the function, you'll get a really confusing and amazing explosion of exception messages and stack trace.  Be advised, on the watch for students that do this.